In [2]:
# https://www.kaggle.com/datasets/ihelon/lego-minifigures-classification/download

--2022-07-11 19:13:18--  https://www.kaggle.com/datasets/ihelon/lego-minifigures-classification/download
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fihelon%2Flego-minifigures-classification%3Fresource%3Ddownload [following]
--2022-07-11 19:13:18--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fihelon%2Flego-minifigures-classification%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download’

download                [ <=>                ]   6.51K  --.-KB/s    in 0s      

2022-07-11 19:13:18 (75.6 MB/s) 

In [ ]:
!python -m pip install -U scikit-image

In [38]:
import random, os, io
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
#from skimage import io, transform
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms

from PIL import Image

In [39]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) 


def ToTensor(image): 
    
    preprocess = transforms.Compose([        
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])
    
    x = preprocess(image)
    x.shape
    
    return {'image': x}

In [40]:
seed_everything() 

lego_data = pd.read_csv('~/Lego/index.csv', index_col=False);lego_data.head()
lego_metadata = pd.read_csv('~/Lego/metadata.csv', index_col=False); lego_metadata.head()

,class_id,lego_ids,lego_names,minifigure_name
0,1,[76115],['Spider Mech vs. Venom'],SPIDER-MAN
1,2,[76115],['Spider Mech vs. Venom'],VENOM
2,3,[76115],['Spider Mech vs. Venom'],AUNT MAY
3,4,[76115],['Spider Mech vs. Venom'],GHOST SPIDER
4,5,[75208],"[""Yoda's Hut""]",YODA


In [41]:
lego_metadata.columns

Index(['class_id', 'lego_ids', 'lego_names', 'minifigure_name'], dtype='object')

In [42]:
lego_data.columns

Index(['path', 'class_id'], dtype='object')

In [44]:
df_lego = pd.merge(lego_data, lego_metadata[['class_id', 'minifigure_name']], on='class_id')
df_lego['labels'] = df_lego['minifigure_name'].apply(lambda x: x.lower())
#df_lego['is_valid'] = df_lego['train-valid'].apply(lambda x:x=='train')
df_lego['fname'] = df_lego['path']; 
df_lego['image'] = df_lego['path'].apply(lambda x: ToTensor(Image.open(x))) 

In [45]:
df_train, df_val = train_test_split(df_lego, test_size=0.2)

In [46]:
df_train

,path,class_id,minifigure_name,labels,fname,image
63,star-wars/0002/012.jpg,6,LUKE SKYWALKER,luke skywalker,star-wars/0002/012.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
324,star-wars/0013/001.jpg,34,DARTH VADER,darth vader,star-wars/0013/001.jpg,"{'image': [[tensor([ 0.8447, 0.8447, 0.8447,..."
355,star-wars/0016/004.jpg,37,OBI-WAN KENOBI,obi-wan kenobi,star-wars/0016/004.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
313,star-wars/0011/006.jpg,32,JANNAH,jannah,star-wars/0011/006.jpg,"{'image': [[tensor([ 0.6734, 0.6392, 0.5707,..."
78,star-wars/0004/003.jpg,8,MACE WINDU,mace windu,star-wars/0004/003.jpg,"{'image': [[tensor([1.3413, 1.3755, 1.3413, 1...."
...,...,...,...,...,...,...
323,star-wars/0012/008.jpg,33,HAN SOLO,han solo,star-wars/0012/008.jpg,"{'image': [[tensor([ 1.4612, 1.4612, 1.4440,..."
192,marvel/0009/001.jpg,21,YELENA BELOVA,yelena belova,marvel/0009/001.jpg,"{'image': [[tensor([ 1.8208, 1.8208, 1.8037,..."
117,star-wars/0008/004.jpg,12,CARA DUNE,cara dune,star-wars/0008/004.jpg,"{'image': [[tensor([0.1083, 0.1254, 0.1083, 0...."
47,star-wars/0001/008.jpg,5,YODA,yoda,star-wars/0001/008.jpg,"{'image': [[tensor([0.5536, 0.5536, 0.5364, 0...."


In [47]:
df_val

,path,class_id,minifigure_name,labels,fname,image
186,marvel/0008/006.jpg,20,BLACK WIDOW,black widow,marvel/0008/006.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
255,jurassic-world/0001/012.jpg,26,OWEN GRADY,owen grady,jurassic-world/0001/012.jpg,"{'image': [[tensor([2.2147, 2.2147, 2.2318, 2...."
45,star-wars/0001/006.jpg,5,YODA,yoda,star-wars/0001/006.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
26,marvel/0003/007.jpg,3,AUNT MAY,aunt may,marvel/0003/007.jpg,"{'image': [[tensor([0.6734, 0.6734, 0.6906, 0...."
216,marvel/0011/003.jpg,23,CAPTAIN AMERICA,captain america,marvel/0011/003.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
...,...,...,...,...,...,...
300,marvel/0017/003.jpg,31,IRON MAN MK 50,iron man mk 50,marvel/0017/003.jpg,"{'image': [[tensor([-2.1179, -2.1179, -2.1179,..."
230,marvel/0012/007.jpg,24,OUTRIDER 1,outrider 1,marvel/0012/007.jpg,"{'image': [[tensor([1.8893, 1.8893, 1.8893, 1...."
167,harry-potter/0001/009.jpg,18,HARRY POTTER,harry potter,harry-potter/0001/009.jpg,"{'image': [[tensor([ 0.0569, -0.4911, -0.9363,..."
116,star-wars/0008/003.jpg,12,CARA DUNE,cara dune,star-wars/0008/003.jpg,"{'image': [[tensor([ 0.7248, 0.7077, 0.6563,..."


In [24]:
train_ds_loader = torch.utils.data.DataLoader(df_train,
                                             batch_size=4, shuffle=True,
                                             num_workers=2)
val_ds_loader = torch.utils.data.DataLoader(df_val,
                                             batch_size=4, shuffle=True,
                                             num_workers=2)